In [1]:
#IMPORT
import os
import json
import shutil
from PIL import Image
import pandas as pd
import re

In [17]:
# READ FUNCTION
def read_and_collect_json_files(directory):
    data = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == 'instance_to_solve.json':
                with open(os.path.join(root, file), 'r') as f:
                    data.append(json.load(f))
    return data

In [18]:
# WRITE FUNCTION
def write_to_json_file(data, output_file):
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)

In [19]:
# MOVE FILES FUNCTION
def move_and_rename_files(src_directory, dst_directory):
    if not os.path.exists(dst_directory):
        os.makedirs(dst_directory)

    i = 0
    for root, dirs, files in os.walk(src_directory):
        for file in files:
            if file == 'instance_to_solve.json':
                src_file_path = os.path.join(root, file)
                dst_file_path = os.path.join(dst_directory, f'instance_to_solve_{i}.json')
                shutil.move(src_file_path, dst_file_path)
                i += 1

In [20]:
# RENAME ITEM NUMBERS
def replace_item_flow(solved_dir, item_data_dir):
    for solved_file in os.listdir(solved_dir):
        if solved_file.startswith('solved_instance') and solved_file.endswith('.json'):
            with open(os.path.join(solved_dir, solved_file), 'r') as f:
                solved_data = json.load(f)
                
            if solved_data.get('status') == 'SAT':
                item_data_file = 'item_data' + solved_file[len('solved_instance'):-5] + '.json'  # corresponding item_data file
                with open(os.path.join(item_data_dir, item_data_file), 'r') as f:
                    item_data = json.load(f)
                
                # replace integers in each sub-list of ITEM_FLOW with corresponding item names
                for sublist in solved_data['ITEM_FLOW']:
                    for i in range(len(sublist)):
                        if sublist[i] == "0":
                            sublist[i] = "none"
                        else:
                            sublist[i] = item_data.get(str(sublist[i]), sublist[i])
                
                # write the updated data back to the file
                with open(os.path.join(solved_dir, solved_file), 'w') as f:
                    json.dump(solved_data, f, indent=4)

In [21]:
# RENAME ASSEMBLER NUMBERS
def replace_assembler(solved_dir, assembler_data_dir):
    for solved_file in os.listdir(solved_dir):
        if solved_file.startswith('solved_instance') and solved_file.endswith('.json'):
            with open(os.path.join(solved_dir, solved_file), 'r') as f:
                solved_data = json.load(f)
                
            if solved_data.get('status') == 'SAT':
                assembler_data_file = 'assembler_data' + solved_file[len('solved_instance'):-5] + '.json'  # corresponding assembler_data file
                with open(os.path.join(assembler_data_dir, assembler_data_file), 'r') as f:
                    assembler_data = json.load(f)
                
                # replace integers in each sub-list of ASSEMBLER with corresponding assembler names
                for sublist in solved_data['ASSEMBLER']:
                    for i in range(len(sublist)):
                        if sublist[i] == "0":
                            sublist[i] = "none"
                        else:
                            sublist[i] = assembler_data[int(sublist[i])-1]
                
                for sublist in solved_data['ASSEMBLER_COLLISION']:
                    for i in range(len(sublist)):
                        if sublist[i] == "0":
                            sublist[i] = "none"
                        else:
                            sublist[i] = assembler_data[int(sublist[i])-1]
                
                # write the updated data back to the file
                with open(os.path.join(solved_dir, solved_file), 'w') as f:
                    json.dump(solved_data, f, indent=4)

In [22]:
# CROP SPRITE SHEET
def crop_sprites(sprite_sheet_path, cols, rows, output_dir):
    # Open the sprite sheet
    sprite_sheet = Image.open(sprite_sheet_path)

    # Calculate the width and height of each sprite
    sprite_width = sprite_sheet.width // cols
    sprite_height = sprite_sheet.height // rows

    # Loop over the sprite sheet and save each sprite
    for row in range(rows):
        for col in range(cols):
            # Calculate the position of the current sprite
            left = col * sprite_width
            top = row * sprite_height
            right = left + sprite_width
            bottom = top + sprite_height

            # Crop the sprite
            sprite = sprite_sheet.crop((left, top, right, bottom))

            # Save the sprite
            sprite.save(f"{output_dir}/sprite_{row}_{col}.png")

In [2]:
def extract_data_and_write_to_excel(main_folder_path, destination_folder):
    # Initialize a dictionary to store the data
    data = {
        "Iteration": [],
        "Optimization Criteria": [],
        "Status": [],
        "Solving Time": []
    }

    # Function to sort file names based on the numerical part
    def sort_key(file_name):
        match = re.search(r'\d+', file_name)
        if match:
            return int(match.group())
        return file_name

    # Iterate over the iterations
    for iteration in sorted(os.listdir(main_folder_path)):
        iteration_path = os.path.join(main_folder_path, iteration)
        
        # Iterate over the optimization criteria folders
        for criteria in sorted(os.listdir(iteration_path)):
            criteria_path = os.path.join(iteration_path, criteria)
            
            # Iterate over the JSON files
            for file in sorted(os.listdir(criteria_path), key=sort_key):
                if file.endswith(".json"):
                    file_path = os.path.join(criteria_path, file)
                    
                    # Open the JSON file and extract the data
                    with open(file_path, "r") as json_file:
                        json_data = json.load(json_file)
                        status = json_data["status"]
                        solving_time = json_data["solving_time"]
                        
                        # Store the data
                        data["Iteration"].append(iteration)
                        data["Optimization Criteria"].append(criteria)
                        data["Status"].append(status)
                        data["Solving Time"].append(solving_time)

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)

    # Write the DataFrame to an Excel file
    output_path = os.path.join(destination_folder, "output2.xlsx")
    df.to_excel(output_path, index=False)

In [29]:
def extract_instance_data_and_write_to_excel(main_folder_path, destination_folder):
    # Initialize a dictionary to store the data
    data_output = {
        "instance": [],
        "height": [],
        "width": [],
        "max_recipes": [],
        "max_items": [],
        "max_assemblers": [],
        "input items": [],
        "input cells": [],
        "output cells": [],
        
    }
    
    # Function to sort file names based on the numerical part
    def sort_key(file_name):
        match = re.search(r'\d+', file_name)
        if match:
            return int(match.group())
        return file_name
    i = 1
    # Iterate over the JSON files
    for file in sorted(os.listdir(main_folder_path), key=sort_key):
        if file.endswith(".json"):
            file_path = os.path.join(main_folder_path, file)
            
            # Open the JSON file and extract the data
            with open(file_path, "r") as json_file:
                data = json.load(json_file)
                
                # Extract the required data
                unique_items = set()
                for recipe in data['recipes'].values():
                    for items in recipe.values():
                        for item in items:
                            unique_items.add(item[0])

                num_unique_items = len(unique_items)
                
                unique_input_items = set()
                for item in data['inOutPos']['IN'].values():
                    unique_input_items.add(item['ITEM'])
                num_unique_input_items = len(unique_input_items)
                
                num_recipes = len(data["recipes"])
                num_unique_items = len(unique_items)
                num_input_cells = len(data["inOutPos"]["IN"])
                num_output_cells = len(data["inOutPos"]["OUT"])
                height = data["size"][0]
                width = data["size"][1]
                
                # Store the data
                data_output["max_recipes"].append(num_recipes)
                data_output["max_items"].append(num_unique_items)
                data_output["input cells"].append(num_input_cells)
                data_output["output cells"].append(num_output_cells)
                data_output["input items"].append(num_unique_input_items)
                data_output["height"].append(height)
                data_output["max_assemblers"].append((width//3)*(height//3))
                data_output["width"].append(width)
                data_output["instance"].append(i)
                i+=1

    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data_output)

    # Write the DataFrame to an Excel file
    output_path = os.path.join(destination_folder, "instance_values.xlsx")
    df.to_excel(output_path, index=False)


In [7]:
# GROUP ALL INSTANCES
directory = 'E:\TFG\solved_instances\instances_to_solve'  # replace with your directory
output_file = 'all_instances.json'  # replace with your output file name
data = read_and_collect_json_files(directory)
write_to_json_file(data, output_file)

In [8]:
# MOVE ALL INSTACES RENAMED
src_directory = 'E:\TFG\solved_instances\instances_to_solve'  # replace with your source directory
dst_directory = 'E:\TFG\solved_instances\instances_to_solve\instances'  # replace with your destination directory
move_and_rename_files(src_directory, dst_directory)

In [11]:
# REPLACE ITEM NAMES
solved_dir = 'E:\TFG\solved_instances\instances_to_solve\solved'  # replace with your solved directory
item_data_dir = 'E:\TFG\solved_instances\instances_to_solve\item_data'  # replace with your item_data directory
replace_item_flow(solved_dir, item_data_dir)

In [23]:
# REPLACE ASSEMBLER NAMES
solved_dir = 'E:\\TFG\\solved_instances\\instances_to_solve\\solved'  # replace with your solved directory
assembler_data_dir = 'E:\\TFG\\solved_instances\\instances_to_solve\\assembler_data'  # replace with your assembler_data directory
replace_assembler(solved_dir, assembler_data_dir)

In [31]:
# CROP IMAGES
crop_sprites("E:\\TFG\\factorio_sprites\\inserter\\hr-inserter-platform.png", 4, 1, "E:\\TFG\\factorio_sprites\\inserter")

In [3]:
# EXPORT TIMES INTO EXCEL
extract_data_and_write_to_excel("E:\TFG\FactorioPlanner\solved_instances", "E:\TFG\instance_times")

In [4]:
# EXPORT INSTANCE DATA TO EXCEL

In [30]:
# Call the function
extract_instance_data_and_write_to_excel("E:\TFG\solved_instances\instances_to_solve\instances", "E:\TFG\instance_times")